# Python Flask REST API Prototype V 1.0  #

*1.보호자 생성* 

*2.보호자 목록*

*3.보호자 검색*

*4.보호자 갱신*


In [1]:
'''
CREATE TABLE guardian (
  num int(11) NOT NULL AUTO_INCREMENT,
  uid varchar(30) NOT NULL,
  name varchar(50) NOT NULL,
  tel varchar(12) NOT NULL,
  birthday varchar(7) NOT NULL,
  pid varchar(255) NOT NULL,
  rdate datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;
END'''

'\nCREATE TABLE guardian (\n  num int(11) NOT NULL AUTO_INCREMENT,\n  uid varchar(30) NOT NULL,\n  name varchar(50) NOT NULL,\n  tel varchar(12) NOT NULL,\n  birthday varchar(7) NOT NULL,\n  pid varchar(255) NOT NULL,\n  rdate datetime DEFAULT NOW(),\n  primary key (num)\n) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;\nEND'

In [2]:
!sudo pip3 install pymysql

     |████████████████████████████████| 51kB 2.4MB/s 


In [3]:
!sudo pip3 install flask-ngrok

In [4]:
!sudo pip3 install flask_restful

     |████████████████████████████████| 51kB 3.3MB/s 


In [5]:
import pymysql
import configparser
from flask import Flask
from datetime import datetime
from google.colab import drive
from flask_restful import reqparse
from flask_restful import Resource, Api
from flask_ngrok import run_with_ngrok

In [6]:
c=drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
CNF_INI = 'drive/My Drive/development/config/openapi.ini'
ERR_URI = 'drive/My Drive/development/test/error.log'

In [8]:
config = configparser.ConfigParser()
config.read(CNF_INI)
HOST = config['mysql']['host']
USER = config['mysql']['user']
PASSWORD = config['mysql']['password']
DATABASE = config['mysql']['database']
PORT = config['mysql']['port']
CHARSET = config['mysql']['charset']

In [9]:
def db_access():
  conn = pymysql.connect(
      host=HOST,
      user=USER,
      password=PASSWORD,
      db=DATABASE,
      charset=CHARSET,
      port=int(PORT),
      cursorclass=pymysql.cursors.DictCursor)
  return conn

In [10]:
def exec_overlap_guardian(conn,uid):
  with conn.cursor() as cursor:
    query = get_overlap_guardian(uid)
    cursor.execute(query)
    for row in cursor:
      data = row.get('cnt')
      
  return data

In [11]:
def exec_create_guardian(conn,uid,name,tel,birthday,pid):
  query = get_create_guardian(uid,name,tel,birthday,pid)
  with conn.cursor() as cursor:
    cursor.execute(query)
  conn.commit()

In [12]:
def exec_list_guardian(conn,limit):
  with conn.cursor() as cursor:
    query = get_list_guardian(limit)
    cursor.execute(query)
    data = cursor.fetchall()
    return data  

In [13]:
def exec_search_guardian(conn,select,keyword):
  with conn.cursor() as cursor:
    query = get_search_guardian(select,keyword)
    cursor.execute(query)
    data = cursor.fetchall()
    return data  

In [14]:
def exec_update_guardian(conn,column,keyword,uid):
  query = get_update_guardian(column,keyword,uid)
  with conn.cursor() as cursor:
    cursor.execute(query)
  conn.commit()

In [15]:
def get_overlap_guardian(uid):
  query = 'select \
  count(*) as cnt \
  from guardian \
  where uid=\'{}\''.format(uid)
  print(query)
  return query

In [16]:
def get_create_guardian(uid,name,tel,birthday,pid):
  query = 'insert into guardian (uid,name,tel,birthday,pid) \
  values (\'{}\',\'{}\',\'{}\',\'{}\',\'{}\')'.format(uid,name,tel,birthday,pid)
  print(query)
  return query

In [17]:
def get_list_guardian(limit):
  query = 'select uid, name, tel, birthday, pid from guardian order by num desc limit {}'.format(limit)
  print(query)
  return query 

In [18]:
def get_search_guardian(select,keyword):
  if select != '' and keyword != '':
    query = 'select uid, name, tel, birthday, pid from guardian where {} like \'%{}%\''.format(select,keyword)
  print(query)
  return query 

In [19]:
def get_update_guardian(column,keyword,uid):
  query = 'update guardian set {}=\'{}\' where uid=\'{}\''.format(column,keyword,uid)
  print(query)
  return query

In [20]:
class confirm_api(Resource):
  def get(self):
    return {'status':'200'}

In [21]:
class create_guardian(Resource):
  def get(self):
    #return {'status':'success'}
    try:
      parser = reqparse.RequestParser()
      parser.add_argument('uid', type=str)
      parser.add_argument('name', type=str)
      parser.add_argument('tel', type=str)
      parser.add_argument('birthday', type=str)
      parser.add_argument('pid', type=str)
      args = parser.parse_args()
      
      uid = args['uid']
      name = args['name']
      tel = args['tel']
      birthday = args['birthday']
      pid = args['pid']

      # print(uid,name,tel,birthday,pid)
      
      conn = db_access()
      
      # print(conn)

      cnt = exec_overlap_guardian(conn,uid)
      
      if cnt:
        return {'status': '100', 'message': 'The guardian exists'}
      else:
        exec_create_guardian(conn,uid,name,tel,birthday,pid)
        return {'staus': '101', 'message': 'You have created guardian. ('+uid+')'}
     
    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))

In [22]:
class list_guardian(Resource):
  def get(self):
    try:
      parser = reqparse.RequestParser()
      parser.add_argument('limit', default=10, type=int)
      args = parser.parse_args()
      
      limit = args['limit']

      # print(limit)
      
      conn = db_access()

      data = exec_list_guardian(conn,limit)

      print(data)
      
      if data:
        return {'staus': '201', 'message': 'I have a guardian', 'data': data}  
      else:
        return {'staus': '200', 'message': 'There are no guardians.'}    

    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))      

*검색 조건 -> 입력 필드에 특수키 '#' 금지*



In [23]:
class search_guardian(Resource):
  def get(self):
    try:
      parser = reqparse.RequestParser()
      parser.add_argument('select', default='uid', type=str)
      parser.add_argument('keyword', default='#', type=str)
      args = parser.parse_args()
      
      select = args['select']
      keyword = args['keyword']
      
      conn = db_access()

      data = exec_search_guardian(conn,select,keyword)

      print(data)
      
      if data:
        return {'staus': '301', 'message': 'I have a guardian', 'data': data}  
      else:
        return {'staus': '300', 'message': 'There are no guardians.'}    

    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))      

*업데이트 조건 -> uid가 null인 경우는 없음*



In [24]:
class update_guardian(Resource):
  def get(self):
    try:
      parser = reqparse.RequestParser()
      parser.add_argument('column', default='name', type=str)
      parser.add_argument('keyword', default='', type=str)
      parser.add_argument('uid', default='', type=str)
      args = parser.parse_args()
      
      column = args['column']
      keyword = args['keyword']
      uid = args['uid']
      
      conn = db_access()

      exec_update_guardian(conn,column,keyword,uid)
      
      if uid:
        return {'staus': '401', 'message': 'The update is complete. ('+uid+')'} 
      else:
        return {'staus': '400', 'message': 'Update failed.'}

    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))      

## Howto ##
보호자 생성 : /guardian/create?uid=test&name=홍길동&tel=01012345678&birthday=xxxxxx1&pid=9876 

보호자 목록 : /guardian/list?limit=5

보호자 검색 : /guardian/search?select=uid&keyword=test 

보호가 갱신 : /guardian/update?column=name&keyword=김영희&uid=test 

In [ ]:
app = Flask(__name__)
api = Api(app)
run_with_ngrok(app)
api.add_resource(confirm_api,'/')
api.add_resource(create_guardian,'/guardian/create/')
api.add_resource(list_guardian,'/guardian/list/')
api.add_resource(search_guardian,'/guardian/search/')
api.add_resource(update_guardian,'/guardian/update/')
app.run()